In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from time import sleep
import json
# from xgboost import XGBRegressor
import sklearn

In [6]:
## Pass in a URL and it returns the data for that professor
def get_data(url):
    req = requests.get(url).content
    bs = BeautifulSoup(req)
    name = list(bs.find("div", class_="NameTitle__Name-dowf0z-0 cfjPUG").descendants)
    name = f"{name[1]} {name[4]}"
    department = bs.find("a", class_="TeacherDepartment__StyledDepartmentLink-fl79e8-0 fuypDB").b.contents[0]
    school = bs.find("a", href="/school?sid=1074").contents[0]
    tags = np.unique(bs.find_all("span", class_="Tag-bs9vf4-0 hHOVKF"), return_counts=True)
    tags = {i:j for i,j in zip(tags[0], tags[1])}
    rating = bs.find("div", class_="RatingValue__Numerator-qw8sqy-2 liyUjw").contents[0]
    num_ratings = bs.find('a', href="#ratingsList").contents[0]
    feedback = bs.find_all("div", "FeedbackItem__FeedbackNumber-uof32n-1 kkESWs")
    take_again = feedback[0].contents[0][:-1]
    difficulty = feedback[1].contents[0]
    return [name, department, school, tags, rating, num_ratings, take_again, difficulty]


In [7]:
## Example dataframe -> appends data to csv file
pd.DataFrame(data=get_data("https://www.ratemyprofessors.com/professor?tid=2466852"), index=["name", "department", "school", "tags", "rating", "take_again", "difficulty"]).transpose().to_csv(f"professors.csv", mode='a', header=False, index=False)

ValueError: Shape of passed values is (8, 1), indices imply (7, 1)

In [8]:
l = json.load(open("uci_rmp_raw_text.json"))["data"]["search"]["teachers"]["edges"]
ids = [l[i]["node"]["legacyId"] for i in range(len(l))] # id's of RMP pages for UCI

In [9]:
## While Loop that checks random pages for professors -> appends data to csv file

for page in ids[3246:]:
    try:
        pd.DataFrame(data=get_data(f"https://www.ratemyprofessors.com/professor?tid={page}"), index=["name", "department", "school", "tags", "rating", "num_ratings", "take_again", "difficulty"]).transpose().to_csv(f"professors.csv", mode='a', header=False, index=False)
        print('x')
    except Exception as e:
        print(e)
    finally:
        sleep(0.1)
        page += 12

x
x
x


KeyboardInterrupt: 

In [16]:
data = pd.read_csv("professors.csv")
import json
for i in range(data.shape[0]):
    print(i)
    data.at[i, "tags"] = {i:j for i,j in data.at[i,"tags"].items() if j > 2}
    

0
{"Clear grading criteria": 2, "Get ready to read": 2, "TEST HEAVY": 1, "Test heavy": 1, "Tough Grader": 1, "Tough grader": 1}
1
{"Amazing lectures ": 1, "Beware of pop quizzes": 1, "Inspirational": 1, "Respected": 1, "Would take again": 1}
2
{"Skip class? You won"t pass.": 3, "Test heavy": 2, "Tough grader": 2}


JSONDecodeError: Expecting ':' delimiter: line 1 column 23 (char 22)

In [ ]:
data

: 

In [ ]:
grades = pd.read_csv("grades.csv")[["instructor", "avg_gpa"]]
query = ~grades["avg_gpa"].isnull()
grades = grades[query]
averages = {
    "instructor": [],
    "avg_gpa": []
}
for i in grades["instructor"].unique():
    averages["avg_gpa"].append(grades[grades.instructor == i].mean().iloc[0])
    averages["instructor"].append(i)


: 

In [17]:
profs = pd.read_csv("professors.csv")
averages = pd.read_csv(("averages.csv"))

In [24]:
averages

,instructor,avg_gpa
0,WILLOUGHBY-HERARD.,3.356250
1,"MALONE, M.",3.915000
2,"EGAN, J.",2.761196
3,"VOGEL, E.",3.390000
4,"DOUGLAS, T.",3.541404
...,...,...
3594,"ZHU, T.",3.290000
3595,"CAIN, D.",3.460000
3596,"REMIGIO, R.",3.550000
3597,"RODRIGUEZ, V.",3.400000


In [20]:
for i in range(profs.shape[0]):
    names = profs.at[i,"name"].split(' ')
    if len(names) > 1:
        profs.at[i,"name"] = f"{names[-1]}, {names[0][0]}.".upper() 
    else:
        profs.at[i,"name"] = i.upper()

In [25]:
profs

,name,department,school,tags,rating,take_again,difficulty
0,"CAMPBELL, M.",Mathematics,UC Irvine,"{'Clear grading criteria': 2, 'Get ready to re...",3.1,0,3.3
1,"MOELLER, B.",Humanities,UC Irvine,"{'Amazing lectures ': 1, 'Beware of pop quizze...",4.2,N/,2.6
2,"APKARIAN, V.",Chemistry,UC Irvine,"{""Skip class? You won't pass."": 3, 'Test heavy...",2.9,40,3.9
3,"MILES, M.",Art History,UC Irvine,"{'Amazing lectures': 1, 'Amazing lectures ': 1...",4.3,80,3.1
4,"GLAZER, A.",Economics,UC Irvine,"{'ACCESSIBLE OUTSIDE CLASS': 1, 'Caring': 1, '...",2.3,0,3.4
...,...,...,...,...,...,...,...
3594,"SHORT, G.",English,UC Irvine,"{'Accessible outside class': 2, 'Amazing lectu...",4.3,62,3.5
3595,"KING, S.",Chemistry,UC Irvine,"{'Accessible outside class': 1, 'Amazing lectu...",4.0,76,3.7
3596,"BERGHOF, A.",Literature,UC Irvine,"{'Accessible outside class': 3, 'Amazing lectu...",3.6,73,2.8
3597,"EDWARDS, K.",Chemistry,UC Irvine,"{'Clear grading criteria': 1, 'Graded by few t...",2.1,12,4.1


In [26]:
profs.sort_values("name").drop_duplicates("name")


,name,department,school,tags,rating,take_again,difficulty
3099,"(NICK), A.",Biological Sciences,UC Irvine,"{'Accessible outside class': 6, 'Amazing lectu...",5.0,99,3.1
2785,", D.",History,UC Irvine,"{'Accessible outside class': 1, 'Amazing lectu...",4.5,67,2.6
2915,", E.",Political Science,UC Irvine,"{'Clear grading criteria': 1, 'Get ready to re...",3.3,100,3.1
2890,", J.",Social Science,UC Irvine,"{'Accessible outside class': 10, 'Amazing lect...",3.9,54,4.3
2588,", M.",Spanish,UC Irvine,"{'Accessible outside class': 1, 'BEWARE OF POP...",4.7,100,3.7
...,...,...,...,...,...,...,...
2382,"ZISSOS, A.",Classics,UC Irvine,"{'Amazing lectures': 4, 'Amazing lectures ': 3...",4.1,56,2.5
497,"ZITO, S.",History,UC Irvine,{},3.8,N/,2.3
3569,"ZIV, H.",Computer Science,UC Irvine,"{'Caring': 2, 'Clear grading criteria': 1, 'Ge...",2.3,43,2.8
3156,"ZOELLER, C.",Sociology,UC Irvine,"{'Accessible outside class': 2, 'Amazing lectu...",4.6,94,2.4


In [28]:
profs

,name,department,school,tags,rating,take_again,difficulty
0,"CAMPBELL, M.",Mathematics,UC Irvine,"{'Clear grading criteria': 2, 'Get ready to re...",3.1,0,3.3
1,"MOELLER, B.",Humanities,UC Irvine,"{'Amazing lectures ': 1, 'Beware of pop quizze...",4.2,N/,2.6
2,"APKARIAN, V.",Chemistry,UC Irvine,"{""Skip class? You won't pass."": 3, 'Test heavy...",2.9,40,3.9
3,"MILES, M.",Art History,UC Irvine,"{'Amazing lectures': 1, 'Amazing lectures ': 1...",4.3,80,3.1
4,"GLAZER, A.",Economics,UC Irvine,"{'ACCESSIBLE OUTSIDE CLASS': 1, 'Caring': 1, '...",2.3,0,3.4
...,...,...,...,...,...,...,...
3594,"SHORT, G.",English,UC Irvine,"{'Accessible outside class': 2, 'Amazing lectu...",4.3,62,3.5
3595,"KING, S.",Chemistry,UC Irvine,"{'Accessible outside class': 1, 'Amazing lectu...",4.0,76,3.7
3596,"BERGHOF, A.",Literature,UC Irvine,"{'Accessible outside class': 3, 'Amazing lectu...",3.6,73,2.8
3597,"EDWARDS, K.",Chemistry,UC Irvine,"{'Clear grading criteria': 1, 'Graded by few t...",2.1,12,4.1


In [31]:
averages = averages.rename(columns={"instructor": "name"})

In [35]:
merged = profs.merge(averages, how="inner", on="name")

In [36]:
merged.to_csv("merged.csv", index=False)

In [27]:
import difflib
def closest_match(value, target_list):
    closest_value = difflib.get_close_matches(value, target_list, n=1, cutoff=0.6)
    if closest_value:
        return closest_value[0]
    else:
        return None

def replace_value(value_list, target_list):
    result = []
    for value in value_list:
        closest = closest_match(value, target_list)
        if closest:
            result.append(closest)
        else:
            result.append(value)
    return result

value_list = list(profs["name"])
target_list = list(averages["instructor"])
result = replace_value(value_list, target_list)
print(result)

['CAMPBELL, M.', 'MOELLER, R.', 'APKARIAN, A.', 'MILES, M.', 'GLAZER, A.', 'MOLZON, W.', 'HESKETH, M.', 'CASTELLANOS, J.', 'SHREIM, S.', 'DOMBRINK, J.', 'STUPAR, J.', 'WANG, T.', 'HOWARD, S.', 'CHAHINIAN, T.', 'TOPIK, S.', 'LENZ, K.', 'MCGUIRE, C.', 'DETWILER, R.', 'REILLY, P.', 'SOBEL, H.', 'NEGEN, J.', 'LIN, K.', 'POWELL, S.', 'BRESNOCK, A.', 'MCKINSON, K.', 'KATO, J.', 'THOMAS, J.', 'MININ, V.', 'PETRACCA, M.', 'NOVIELLO, N.', 'JESILOW, P.', 'CHOU, P.', 'FAMIGLIETTI, C.', 'DABDUB, D.', 'SNOW, D.', 'TOBIAS, D.', 'FREEMAN, F.', 'YU, J.', 'FRANZ, M.', 'REZAEI, A.', 'BERKELHAMER, R.', 'IBER, L.', 'KOO, H.', 'UBAN, K.', 'BUOTE, D.', 'ROSE, M.', 'DANDAMUDI, K.', 'MAURER, W.', 'JENKS, A.', 'ZHAO, H.', 'PASTOR, J.', 'JOHNSON, V.', 'SALEN, K.', 'SYMANSKI, R.', 'SAFARZADEH, M.', 'STERN, S.', 'KLEINFELDER, S.', 'ZHENG, X.', 'HEYDARI, P.', 'LIU, J.', 'KROLL, J.', 'SILVER, V.', 'AWAD, S.', 'OSBORN, B.', 'MEEKER, J.', 'MIMURA, G.', 'TITA, G.', 'VIGIL, V.', 'NAVA, A.', 'MIN, Y.', 'LEUNG, S.', 'LEE

: 